In [ ]:
%reload_ext autoreload
%autoreload 2

#%matplotlib notebook
%matplotlib inline

from math import pi
import numpy as np
import matplotlib.pylab as plt

import gtsam

# Pose2SLAM example

Adopted from `python/gtsam/tests/test_Pose2SLAMExample.py`

In [ ]:
import plot

graph = gtsam.NonlinearFactorGraph()

# prior
priorMean = gtsam.Pose2(0.0, 0.0, 0.0)  # prior at origin
priorNoise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
graph.add(gtsam.PriorFactorPose2(1, priorMean, priorNoise))

# odometry
odometryNoise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
graph.add(
    gtsam.BetweenFactorPose2(1, 2, gtsam.Pose2(2.0, 0.0, 0.0), odometryNoise))
graph.add(
    gtsam.BetweenFactorPose2(2, 3, gtsam.Pose2(2.0, 0.0, pi / 2),
                             odometryNoise))
graph.add(
    gtsam.BetweenFactorPose2(3, 4, gtsam.Pose2(2.0, 0.0, pi / 2),
                             odometryNoise))
graph.add(
    gtsam.BetweenFactorPose2(4, 5, gtsam.Pose2(2.0, 0.0, pi / 2),
                             odometryNoise))

# loop closure
model = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
graph.add(gtsam.BetweenFactorPose2(5, 2, gtsam.Pose2(2.0, 0.0, pi / 2), model))

# initialization
initialEstimate = gtsam.Values()
initialEstimate.insert(1, gtsam.Pose2(0.5, 0.0, 0.2))
initialEstimate.insert(2, gtsam.Pose2(2.3, 0.1, -0.2))
initialEstimate.insert(3, gtsam.Pose2(4.1, 0.1, pi / 2))
initialEstimate.insert(4, gtsam.Pose2(4.0, 2.0, pi))
initialEstimate.insert(5, gtsam.Pose2(2.1, 2.1, -pi / 2))

# Optimize using Levenberg-Marquardt optimization with an ordering from colamd
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initialEstimate)
result = optimizer.optimizeSafely()

# plot results
marginals = gtsam.Marginals(graph, result)
plt.figure()
for i in range(1, 6):
    result.atPose2(i)
    #x, y, angle = result.atPose2(i).x(), result.atPose2(i).y(), result.atPose2(
    #    i).theta()
    P = marginals.marginalCovariance(i)
    #plt.scatter(x, y, s=50, label=f'pose {i}')
    #plt.arrow(x, y, np.cos(angle), np.sin(angle), width=0.01)
    plot.plot_pose2(
        1,
        pose=result.atPose2(i),
        covariance=P,
    )
plt.axis('equal')
#plt.legend(loc='upper left')
plt.grid()

# SFM example

Adopted from `http://docs.ros.org/en/melodic/api/gtsam/html/SFMExample_8py_source.html`

In [ ]:
from gtsam import symbol_shorthand
L = symbol_shorthand.L
X = symbol_shorthand.X

from gtsam.examples import SFMdata
from gtsam import (Cal3_S2, DoglegOptimizer,
                         GenericProjectionFactorCal3_S2, Marginals,
                         NonlinearFactorGraph, PinholeCameraCal3_S2, Point3,
                         Pose3, PriorFactorPoint3, PriorFactorPose3, Rot3, Values)

"""
Camera observations of landmarks (i.e. pixel coordinates) will be stored as Point2 (x, y).

Each variable in the system (poses and landmarks) must be identified with a unique key.
We can either use simple integer keys (1, 2, 3, ...) or symbols (X1, X2, L1).
Here we will use Symbols

In GTSAM, measurement functions are represented as 'factors'. Several common factors
have been provided with the library for solving robotics/SLAM/Bundle Adjustment problems.
Here we will use Projection factors to model the camera's landmark observations.
Also, we will initialize the robot at some location using a Prior factor.

When the factors are created, we will add them to a Factor Graph. As the factors we are using
are nonlinear factors, we will need a Nonlinear Factor Graph.

Finally, once all of the factors have been added to our factor graph, we will want to
solve/optimize to graph to find the best (Maximum A Posteriori) set of variable values.
GTSAM includes several nonlinear optimizers to perform this step. Here we will use a
trust-region method known as Powell's Degleg

The nonlinear solvers within GTSAM are iterative solvers, meaning they linearize the
nonlinear functions around an initial linearization point, then solve the linear system
to update the linearization point. This happens repeatedly until the solver converges
to a consistent set of variable values. This requires us to specify an initial guess
for each variable, held in a Values container.
"""

# Define the camera calibration parameters
K = Cal3_S2(50.0, 50.0, 0.0, 50.0, 50.0)

# Define the camera observation noise model
measurement_noise = gtsam.noiseModel.Isotropic.Sigma(2, 1.0)  # one pixel in u and v

# Create the set of ground-truth landmarks
points = SFMdata.createPoints()

# Create the set of ground-truth poses
poses = SFMdata.createPoses(K)

# Create a factor graph
graph = NonlinearFactorGraph()

# Add a prior on pose x1. This indirectly specifies where the origin is.
# 0.3 rad std on roll,pitch,yaw and 0.1m on x,y,z
pose_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.3, 0.1, 0.1, 0.1]))
factor = PriorFactorPose3(X(0), poses[0], pose_noise)
graph.push_back(factor)

# Simulated measurements from each camera pose, adding them to the factor graph
for i, pose in enumerate(poses):
    camera = PinholeCameraCal3_S2(pose, K)
    for j, point in enumerate(points):
        measurement = camera.project(point)
        factor = GenericProjectionFactorCal3_S2(
            measurement, measurement_noise, X(i), L(j), K)
        graph.push_back(factor)

# Because the structure-from-motion problem has a scale ambiguity, the problem is still under-constrained
# Here we add a prior on the position of the first landmark. This fixes the scale by indicating the distance
# between the first camera and the first landmark. All other landmark positions are interpreted using this scale.
point_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.1)
factor = PriorFactorPoint3(L(0), points[0], point_noise)
graph.push_back(factor)
#graph.print('Factor Graph:\n')

# Create the data structure to hold the initial estimate to the solution
# Intentionally initialize the variables off from the ground truth
initial_estimate = Values()
for i, pose in enumerate(poses):
    transformed_pose = pose.retract(0.1*np.random.randn(6,1))
    initial_estimate.insert(X(i), transformed_pose)
for j, point in enumerate(points):
    transformed_point = point + 0.1*np.random.randn(3)
    initial_estimate.insert(L(j), transformed_point)
#initial_estimate.print('Initial Estimates:\n')

# Optimize the graph and print results
params = gtsam.DoglegParams()
#params.setVerbosity('TERMINATION')
params.setVerbosity('')
optimizer = DoglegOptimizer(graph, initial_estimate, params)
print('Optimizing:')
result = optimizer.optimize()
#result.print('Final results:\n')
print('initial error = {}'.format(graph.error(initial_estimate)))
print('final error = {}'.format(graph.error(result)))

print(result)

marginals = Marginals(graph, result)
fig = plt.figure()
plot.plot_3d_points(1, result, marginals=marginals, cov_scale=0.1)
plot.plot_trajectory(1, result, cov_scale=0.1, marginals=marginals, axis_length=8)
plot.set_axes_equal(1)
fig.set_size_inches(10, 10)

# Planar SLAM example

In [ ]:
# create plane measurement setup
from gtsam import Unit3, Pose2
from gtsam import OrientedPlane3 as Plane

X = symbol_shorthand.X
P = symbol_shorthand.P

# velocity 
v = [0, 0.1, 0]; sigma_v = 0.01 # m/s
w = [0, 0, 0]; sigma_w = 5 * np.pi / 180 # rad/s (roll, pitch, yaw)

poses = []
planes_local = []

# pose
R = Rot3()
t = Point3(0.0, 0.0, 0.0)
pose_t = Pose3(r=R, t=t)   

# plane
d_0 = 1.0
n_0 = [0, 1.0, 0]
unit_0 = Unit3(n_0) # unit vector
plane_t = Plane(n=unit_0, d=d_0) # one meter

planes_local.append(plane_t)
poses.append(pose_t)

fig = plt.figure(0)
fig.set_size_inches(10, 10)

delta_t = 1
for t in range(5):
    v_noisy = delta_t * (v + np.random.normal(size=len(v), scale=sigma_v))
    w_noisy = delta_t * (w + np.random.normal(size=len(w), scale=sigma_w))
    
    R_delta = Rot3.Ypr(*w_noisy)
    t_delta = Point3(v_noisy)
    pose_delta = Pose3(r=R_delta, t=t_delta)
    
    pose_t = pose_delta * pose_t
    plane_t = plane_t.transform(pose_delta.inverse())
    print("new distance:", plane_t.distance())
    
    R = R * R_delta
    t = t + t_delta
    
    estimate = Values()
    estimate.insert(X(1), pose_t)
    estimate.insert(P(1), plane_t)

    # TODO(continue here) plotting absolute wall normals doesn't make sense, should anchor them
    # to current estimate! 
    plot.plot_trajectory(0, estimate, axis_length=0.2)
plot.set_axes_equal(0)
plt.gca().view_init(elev=0., azim=0)
plt.show()

In [ ]:
from gtsam import symbol_shorthand
from gtsam import DoglegOptimizer
from gtsam import OrientedPlane3 as Plane
from gtsam import OrientedPlane3Factor as PlaneFactor
from gtsam import NonlinearFactorGraph
from gtsam import Values
from gtsam import Pose3, Rot3, Point3
from gtsam import PriorFactorOrientedPlane3 # is created automatically!

X = symbol_shorthand.X # Pose3 (x, y, z, r, p, y)
P = symbol_shorthand.P # Plane 

# Create a factor graph
graph = NonlinearFactorGraph()
initial_estimate = Values()

#Initial values
p3 = Plane(np.array([0.211098835, 0.214292752, 0.95368543, 26.4269514]))
p2 = Plane(np.array([0.301901811, 0.151751467, 0.941183717, 33.4388229]))

rot3 = Rot3(np.array([0.799903913, -0.564527097,  0.203624376, 0.552537226,   0.82520195,  0.117236322, -0.234214312, 0.0187322547,  0.972004505]))
point3 = Point3(np.array([-91.7500013,-0.47569366,-4.61067606]))
x1 = Pose3(rot3, point3)

#Setup priors
initial_estimate = Values()
initial_estimate.insert(X(1), x1)
initial_estimate.insert(P(2), p2)
initial_estimate.insert(P(3), p3)

#Setup prior factors
x1_prior = Pose3(rot3, point3)
x1_noise = gtsam.noiseModel.Isotropic.Sigma(6, 0.01)  # one pixel in u and v
factor = PriorFactorPose3(X(1), x1_prior, x1_noise)
graph.push_back(factor)

p3_prior = p3
p2_prior = p2

p3_noise = gtsam.noiseModel.Diagonal.Sigmas([0.785398163, 0.785398163, 5])
p2_noise = gtsam.noiseModel.Diagonal.Sigmas([0.785398163, 0.785398163, 5])

factor = PriorFactorOrientedPlane3(P(3), p3_prior, p3_noise)
graph.push_back(factor)
factor = PriorFactorOrientedPlane3(P(2), p2_prior, p2_noise)
graph.push_back(factor)

# Setup plane factors
p3_meas = Plane(np.array([0.0638967294, 0.0755284627, 0.995094297, 2.55956073]))
p2_meas = Plane(np.array([0.104902077, -0.0275756528, 0.994100165, 1.32765088]))
x1_p2_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.0322889);
x1_p3_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.0451801);

#factor = PlaneFactor(p2_meas.planeCoefficients(), x1_p2_noise, X(1), P(2))
#graph.push_back(factor)

#factor = PlaneFactor(p3_meas.planeCoefficients(), x1_p3_noise, X(1), P(3))
#graph.push_back(factor)

# Optimize the graph and print results
params = gtsam.DoglegParams()
params.setVerbosity('')
optimizer = DoglegOptimizer(graph, initial_estimate, params)
print('Optimizing:')
result = optimizer.optimize()
print('initial error = {}'.format(graph.error(initial_estimate)))
print('final error = {}'.format(graph.error(result)))

#marginals = Marginals(graph, result)
#fig = plt.figure()
#plot.plot_3d_points(1, result, marginals=marginals, cov_scale=0.1)
#plot.plot_trajectory(1, result, cov_scale=0.1, marginals=marginals, axis_length=8)
#plot.set_axes_equal(1)
#fig.set_size_inches(10, 10)